# Dataset Statistics

In [37]:
#Import required libraries
import torch
import torch.nn as nn
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch_snippets import stems, read
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np

In [14]:
def get_transforms():
  # Dataset is large, don't apply random data augmentation
  # Mean and std dev based on average of training data
  return transforms.Compose([
                             transforms.ToTensor(),
                             ])

In [15]:
# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device being used is:", device)

The device being used is: cpu


In [16]:
class SegmentationDataset(Dataset):
    def __init__(self, split):
        self.items = stems(f'dataset/image_{split}')
        self.split = split


    def __len__(self):
        return len(self.items)


    def __getitem__(self, ix):
        image = read(f'dataset/image_{self.split}/{self.items[ix]}.png', 1)
        image = cv2.resize(image, (572,572))

        mask = read(f'dataset/mask_{self.split}/{self.items[ix]}.png')
        mask = cv2.resize(mask, (572,572))

        return image, mask
    
    def collate_fn(self, batch):
        ims, masks = list(zip(*batch))

        ims = torch.cat([get_transforms()(im.copy()/255.)[None] for im in ims]).float().to(device)


        ce_masks = torch.cat([torch.Tensor(mask)[None] for mask in masks]).long().to(device)


        return ims, ce_masks

In [17]:
trn_ds = SegmentationDataset('train')
val_ds = SegmentationDataset('val')
test_ds = SegmentationDataset('test')

trn_dl = DataLoader(trn_ds, batch_size=16, shuffle=False, collate_fn=trn_ds.collate_fn)
val_dl = DataLoader(val_ds, batch_size=16, collate_fn=val_ds.collate_fn)
test_dl = DataLoader(test_ds, batch_size=16, collate_fn=test_ds.collate_fn)

### Calculate the Label Distribution for Each Set

In [57]:
train_count = np.zeros(34)
val_count = np.zeros(34)
test_count = np.zeros(34)

for i in range(len(trn_ds)):
    image, mask = trn_ds[i]
    train_count += np.bincount(mask.flatten(), minlength = 34)

print("Pixel count in the training set:", train_count)

for i in range(len(val_ds)):
    image, mask = val_ds[i]
    val_count += np.bincount(mask.flatten(), minlength = 34)

print("Pixel count in the validation set:", val_count)

for i in range(len(test_ds)):
    image, mask = test_ds[i]
    test_count += np.bincount(mask.flatten(), minlength = 34)

print("Pixel count in the testing set:", train_count)


Pixel count in the training set: [8.28760000e+04 3.77709270e+07 1.24280910e+07 1.28947780e+07
 1.10625570e+07 2.48205700e+06 9.83742800e+06 2.63813555e+08
 4.18213870e+07 5.66634200e+06 2.40685400e+06 1.63464176e+08
 4.54986800e+06 6.48666600e+06 5.09433000e+05 3.15337100e+06
 8.67755000e+05 8.80669900e+06 6.05012000e+05 2.04127000e+06
 4.47253700e+06 1.06123604e+08 8.20591200e+06 2.86622730e+07
 8.91648500e+06 1.02424900e+06 5.10874750e+07 2.07781700e+06
 1.76852300e+06 3.53082000e+05 1.87956000e+05 1.51118900e+06
 6.34511000e+05 3.02213300e+06]
Pixel count in the validation set: [2.0151000e+04 6.2846330e+06 2.3860000e+04 2.6127850e+06 1.7382940e+06
 5.6550500e+05 2.4502190e+06 5.3220787e+07 1.0220493e+07 6.6270900e+05
 1.2315100e+05 3.1171523e+07 1.6043050e+06 1.4425310e+06 1.0651600e+05
 1.1897200e+05 1.8584100e+05 1.2838680e+06 1.0586600e+05 3.6734500e+05
 7.2316600e+05 2.9840045e+07 1.9669170e+06 5.9002710e+06 1.3740520e+06
 2.6590400e+05 8.4745950e+06 2.3592900e+05 2.6326300e+05 

### Calculate Mean Training Dataset Staistics for Normalization

In [46]:
from tqdm import tqdm

mean = np.array([0.00000, 0.00000, 0.00000])
std = np.array([0.00000, 0.00000, 0.00000])
for idx, data in tqdm(enumerate(trn_dl), total = len(trn_dl)):
    ims, ce_masks = data

    mean[0] += ims[:, 0].mean().item()
    std[0] += ims[:, 0].std().item()
    
    mean[1] += ims[:, 1].mean().item()
    std[1] += ims[:, 1].std().item()
    
    mean[2] += ims[:, 2].mean().item()
    std[2] += ims[:, 2].std().item()
    
    print(idx+1)
    print("Running mean:", mean / (idx+1))
    print("Running std dev:", std / (idx+1))

  1%|          | 1/155 [00:01<04:16,  1.66s/it]

1
Running mean: [0.23786305 0.27982879 0.23795477]
Running std dev: [0.1413877  0.15027779 0.14686461]


  1%|▏         | 2/155 [00:03<03:59,  1.56s/it]

2
Running mean: [0.23480281 0.27548064 0.23559354]
Running std dev: [0.14495707 0.15336681 0.14899515]


  2%|▏         | 3/155 [00:04<03:54,  1.54s/it]

...

153
Running mean: [0.28493145 0.32220478 0.28132807]
Running std dev: [0.17576599 0.18084703 0.17809844]


 99%|█████████▉| 154/155 [04:12<00:01,  1.80s/it]

154
Running mean: [0.28494095 0.32225396 0.28137361]
Running std dev: [0.17573603 0.18080828 0.17806175]


100%|██████████| 155/155 [04:13<00:00,  1.64s/it]

155
Running mean: [0.28509203 0.32244492 0.2815879 ]
Running std dev: [0.17581021 0.18087702 0.17814998]
